In [1]:
import pandas as pd
import json
from semantic_text_splitter import TextSplitter
from tqdm import tqdm


def reconstruct_paper(example: pd.Series) -> str:
    return f"{example['title']}\n\nAbstract: {example['abstract']}\n\n{example['body']}"

In [ ]:
with open("data/research_dois_used.json", "r") as f:
    research_dois_used = json.load(f)

print(len(research_dois_used))
research_doi_set = set(research_dois_used)
print(len(research_doi_set))

9989
9989


In [ ]:
research_df = pd.read_json("data/research_used.jsonl", lines=True)
print(f"Unique dois in research used: {len(research_df.doi.unique())}")

Unique dois in research used: 9989


In [ ]:
# Confirm we have exactly the doi's expected
research_dois = set(research_df.doi)
research_dois == research_doi_set

True

In [7]:
splitter = TextSplitter(capacity=1500, overlap=150)
chunk_records = []
for doi in tqdm(research_dois):
    record = research_df[research_df.doi == doi].iloc[0]
    paper = reconstruct_paper(record)
    chunks = splitter.chunks(paper)
    chunks = [chunk.strip().replace("\x00", "") for chunk in chunks if chunk.strip()]  # Remove null chars and empty chunks
    pubdate = int(record.pubdate.replace("-", ""))
    citation_count = int(record.citation_count)
    for chunk in chunks:
        chunk_records.append(
            {
                "doi": doi, 
                "text": chunk,
                "citation_count": citation_count,
                "pubdate": pubdate,
            }
        )

chunks_df = pd.DataFrame(chunk_records)
chunks_df.to_json("data/research_chunks.jsonl", lines=True, orient="records")

100%|██████████| 9989/9989 [00:10<00:00, 939.82it/s] 


In [8]:
chunks_df.head()

,doi,text,citation_count,pubdate
0,10.1086/311318,Detection of Abundant CO<SUB>2</SUB> Ice in th...,154,19980501
1,10.1086/311318,Abstract: We report the first detection of sol...,154,19980501
2,10.1086/311318,<P />Based on observations with Infrared Space...,154,19980501
3,10.1086/311318,1. INTRODUCTION Icy mantles are an important c...,154,19980501
4,10.1086/311318,1986 ; Sandford et al. 1988 ; Bernstein et al....,154,19980501


In [9]:
len(chunks_df)

460801